# Phase 2B: 원본 소스 데이터 검증 노트북

이 노트북은 Knowledge Base 구축을 위한 원본 소스 데이터를 검증하고 통합합니다.

## 검증 대상 소스
1. **미쉐린 가이드**: https://guide.michelin.com/kr/ko
2. **블루리본**: https://bluer.co.kr
3. **셰프 인터뷰**: 언론 기사, 매거진, 유튜브 등
4. **코스 구성**: 공식 웹사이트, 블로그 리뷰 등
5. **브랜드 철학/공간**: 공식 웹사이트, 언론 보도 자료 등

## 데이터 형식
```json
{
  "shop_seq": 1,
  "shop_name": "모수",
  "category": "fine_dining",
  "sources": [
    {
      "source_type": "michelin_review",
      "text": "...",
      "url": "https://...",
      "collected_date": "2025-01-11"
    },
    ...
  ]
}
```

## 사용 방법
1. 섹션 1-3: 환경 설정 (최초 1회)
2. 섹션 4: 검증 데이터 입력 (source_generator.ipynb 출력 또는 수동 입력)
3. 섹션 5: 데이터 검증
4. 섹션 6: JSON 파일로 저장

## 섹션 1: 패키지 설치

In [ ]:
%pip install pandas python-dotenv

## 섹션 2: 라이브러리 Import

In [ ]:
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict

print("✅ 라이브러리 Import 완료")

## 섹션 3: 데이터 스키마 정의

In [ ]:
# ========================================
# 데이터 스키마
# ========================================

VALID_SOURCE_TYPES = [
    "michelin_review",
    "blueribbon_review",
    "chef_interview",
    "course_description",
    "brand_philosophy",
    "space_ambiance",
    "tradition_technique"
]

VALID_CATEGORIES = [
    "fine_dining",
    "mid_price",
    "waiting_hotplace"
]

def validate_source_data(data: Dict) -> tuple[bool, str]:
    """
    소스 데이터 유효성 검증
    
    Returns:
        (is_valid, error_message)
    """
    # 필수 필드 확인
    required_fields = ["shop_seq", "shop_name", "category", "sources"]
    for field in required_fields:
        if field not in data:
            return False, f"필수 필드 누락: {field}"
    
    # 카테고리 확인
    if data["category"] not in VALID_CATEGORIES:
        return False, f"유효하지 않은 카테고리: {data['category']}"
    
    # sources 리스트 확인
    if not isinstance(data["sources"], list):
        return False, "sources는 리스트여야 합니다"
    
    if len(data["sources"]) == 0:
        return False, "최소 1개 이상의 소스가 필요합니다"
    
    # 각 소스 검증
    for idx, source in enumerate(data["sources"]):
        # 필수 필드
        source_required = ["source_type", "text"]
        for field in source_required:
            if field not in source:
                return False, f"소스 {idx}번: 필수 필드 누락: {field}"
        
        # source_type 확인
        if source["source_type"] not in VALID_SOURCE_TYPES:
            return False, f"소스 {idx}번: 유효하지 않은 source_type: {source['source_type']}"
        
        # 텍스트 길이 확인
        if len(source["text"].strip()) < 50:
            return False, f"소스 {idx}번: 텍스트가 너무 짧습니다 (최소 50자)"
    
    return True, "검증 통과"


print("✅ 데이터 스키마 정의 완료")
print(f"   - 유효한 소스 타입: {', '.join(VALID_SOURCE_TYPES)}")
print(f"   - 유효한 카테고리: {', '.join(VALID_CATEGORIES)}")

## 섹션 4: 원본 소스 데이터 입력

source_generator.ipynb에서 생성된 JSON 데이터 또는 수동으로 수집한 데이터를 입력하세요.

In [ ]:
# ========================================
# 원본 소스 데이터 입력
# ========================================

# 예시 데이터 (실제 수집한 데이터로 교체하세요)
COLLECTED_SOURCES = [
    {
        "shop_seq": 1,
        "shop_name": "모수",
        "category": "fine_dining",
        "sources": [
            {
                "source_type": "michelin_review",
                "text": """모수는 안성재 셰프가 운영하는 모던 한식 레스토랑입니다. 화덕 구이 기법을 활용한 창의적인 한식을 선보이며, 제철 식재료를 중심으로 한국의 전통 조리법에 현대적 감각을 더했습니다. 깔끔하고 모던한 공간에서 셰프의 철학이 담긴 정교한 플레이팅을 경험할 수 있습니다. 미쉐린 2스타를 유지하며 한국 파인다이닝의 새로운 기준을 제시하고 있습니다.""",
                "url": "https://guide.michelin.com/kr/ko/seoul-region/seoul/restaurant/mosu",
                "collected_date": "2025-01-11"
            },
            {
                "source_type": "chef_interview",
                "text": """Q: 모수의 요리 철학은 무엇인가요?
A: 저는 한국 식재료에 대한 깊은 이해와 존중을 바탕으로 요리합니다. 화덕을 중심으로 한 조리 방식은 한국의 전통 아궁이에서 영감을 받았습니다. 불의 온도와 연기가 식재료에 미치는 영향을 연구하며, 이를 통해 한국 식재료의 본질을 끌어냅니다.

Q: 가장 중요하게 생각하는 가치는?
A: 현지성(Locality)입니다. 한국에서만 맛볼 수 있는, 이곳에서만 가능한 요리를 만들고 싶습니다. 그래서 제철 식재료를 고집하고, 전통 발효 기법을 현대적으로 재해석합니다.""",
                "url": "https://example.com/interview/ahn-sung-jae",
                "collected_date": "2025-01-11"
            },
            {
                "source_type": "course_description",
                "text": """모수의 시그니처 코스는 10-12가지 요리로 구성됩니다. 매 시즌 메뉴가 바뀌며, 제철 식재료를 중심으로 화덕 구이 기법이 적용됩니다. 대표 메뉴로는 '화덕에 구운 제철 채소', '한우 숙성육 화덕 구이', '발효 소스를 곁들인 해산물' 등이 있습니다.""",
                "url": "https://example.com/mosu/menu",
                "collected_date": "2025-01-11"
            }
        ]
    },
    {
        "shop_seq": 2,
        "shop_name": "밍글스",
        "category": "fine_dining",
        "sources": [
            {
                "source_type": "michelin_review",
                "text": """밍글스는 강민구 셰프가 선보이는 창의적인 한식 파인다이닝입니다. 한국 전통 식재료와 발효 기법을 현대적으로 재해석하며, 특히 장류(된장, 간장, 고추장)를 활용한 혁신적인 요리로 유명합니다. 미쉐린 2스타를 획득하며 세계적으로 한국 요리의 가능성을 입증했습니다.""",
                "url": "https://guide.michelin.com/kr/ko/seoul-region/seoul/restaurant/mingles",
                "collected_date": "2025-01-11"
            },
            {
                "source_type": "brand_philosophy",
                "text": """밍글스는 '한국의 맛을 현대적으로 재해석한다'는 철학을 가지고 있습니다. 'Mingle'은 '섞이다, 어우러지다'는 의미로, 전통과 현대, 동양과 서양, 익숙함과 낯섦이 조화롭게 어우러지는 경험을 제공합니다. 한국인에게는 새로운 발견을, 외국인에게는 한국 미식의 진수를 전달하는 것이 목표입니다.""",
                "url": "https://example.com/mingles/philosophy",
                "collected_date": "2025-01-11"
            }
        ]
    }
    # 추가 매장 데이터를 여기에 입력하세요...
]

print(f"📥 수집된 데이터: {len(COLLECTED_SOURCES)}개 매장")
for shop_data in COLLECTED_SOURCES:
    num_sources = len(shop_data["sources"])
    total_chars = sum(len(s["text"]) for s in shop_data["sources"])
    print(f"   - {shop_data['shop_name']}: {num_sources}개 소스, 총 {total_chars:,}자")

## 섹션 5: 데이터 검증

In [ ]:
# ========================================
# 데이터 검증
# ========================================

print("🔍 데이터 검증 중...\n")
print("=" * 80)

validation_results = []
all_valid = True

for idx, shop_data in enumerate(COLLECTED_SOURCES, 1):
    shop_name = shop_data.get("shop_name", f"매장{idx}")
    
    print(f"\n[{idx}] {shop_name}")
    print("─" * 80)
    
    is_valid, message = validate_source_data(shop_data)
    
    if is_valid:
        print(f"✅ {message}")
        
        # 상세 정보
        num_sources = len(shop_data["sources"])
        total_chars = sum(len(s["text"]) for s in shop_data["sources"])
        source_types = [s["source_type"] for s in shop_data["sources"]]
        
        print(f"   - 카테고리: {shop_data['category']}")
        print(f"   - 소스 개수: {num_sources}개")
        print(f"   - 총 텍스트: {total_chars:,}자")
        print(f"   - 소스 타입: {', '.join(source_types)}")
        
        validation_results.append({
            "shop_name": shop_name,
            "valid": True,
            "num_sources": num_sources,
            "total_chars": total_chars
        })
    else:
        print(f"❌ {message}")
        all_valid = False
        
        validation_results.append({
            "shop_name": shop_name,
            "valid": False,
            "error": message
        })

print("\n" + "=" * 80)

if all_valid:
    print("\n✅ 모든 데이터 검증 통과")
    print(f"   - 총 {len(COLLECTED_SOURCES)}개 매장")
    print(f"   - 총 {sum(r['num_sources'] for r in validation_results)}개 소스")
    print(f"   - 총 {sum(r['total_chars'] for r in validation_results):,}자")
else:
    print("\n❌ 일부 데이터에 오류가 있습니다. 위의 메시지를 확인하세요.")

## 섹션 6: JSON 파일로 저장

In [ ]:
# ========================================
# JSON 파일로 저장
# ========================================

if all_valid:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"collected_sources_{timestamp}.json"
    
    # JSON 저장
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(COLLECTED_SOURCES, f, ensure_ascii=False, indent=2)
    
    print(f"💾 데이터 저장 완료: {filename}")
    print(f"   - 크기: {len(json.dumps(COLLECTED_SOURCES, ensure_ascii=False)):,} bytes")
    
    # 통계 CSV도 생성
    df_stats = pd.DataFrame(validation_results)
    csv_filename = f"collected_sources_stats_{timestamp}.csv"
    df_stats.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    
    print(f"📊 통계 파일 저장: {csv_filename}")
    
    print("\n✅ 다음 단계: source_indexer.ipynb에서 이 데이터를 벡터 DB에 인덱싱합니다.")
else:
    print("⚠️ 검증 실패로 저장하지 않습니다. 데이터를 수정 후 다시 실행하세요.")

## 섹션 7: 카테고리별 통계 (선택)

In [ ]:
# 카테고리별 통계
if all_valid:
    print("📊 카테고리별 통계\n")
    print("=" * 80)
    
    for category in VALID_CATEGORIES:
        category_data = [s for s in COLLECTED_SOURCES if s["category"] == category]
        
        if category_data:
            print(f"\n📌 {category}")
            print("─" * 80)
            print(f"   - 매장 수: {len(category_data)}개")
            
            total_sources = sum(len(s["sources"]) for s in category_data)
            print(f"   - 총 소스: {total_sources}개")
            
            # 소스 타입별 개수
            source_type_counts = {}
            for shop in category_data:
                for source in shop["sources"]:
                    stype = source["source_type"]
                    source_type_counts[stype] = source_type_counts.get(stype, 0) + 1
            
            print(f"   - 소스 타입별:")
            for stype, count in sorted(source_type_counts.items()):
                print(f"      • {stype}: {count}개")
    
    print("\n" + "=" * 80)

## 다음 단계

이제 검증된 데이터를 `source_indexer.ipynb`에서 청킹하고 벡터 DB에 인덱싱합니다.

### 워크플로우
1. ✅ **source_generator.ipynb** - 원본 소스 자동 생성
2. ✅ **source_validator.ipynb** (현재) - 데이터 검증 및 통합
3. ⏭️ **source_indexer.ipynb** - 벡터 DB 인덱싱
4. ⏭️ **main_rag_source.ipynb** - Source-based RAG
5. ⏭️ **main_rag_hybrid.ipynb** - Hybrid RAG